In [251]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

df = pd.read_csv('/Users/seanyu_mac/Downloads/public_daily_access_log_export_2025-02-14_110229.csv')
df

,id,user_id,device_id,access_date,created_at
0,1,c8313ed7-1dc9-4267-a9cd-f5ab98cfa03d,994087da-b2fc-4e4a-b1fb-b7d9f7689161,2024-10-07,2024-10-07 00:21:40.801
1,2,947e597c-776f-43d8-bf83-5724faae61f7,6cd11f67-bf2a-4a5a-8871-7c59b2dbbc4e,2024-10-07,2024-10-07 00:23:42.744
2,3,c3f1092d-2028-4d25-b533-ff4d6caf21ac,f58afcf3-8d51-4a3c-86e9-09d48c28b0d6,2024-10-07,2024-10-07 01:06:35.559
3,4,a51fdaca-f9eb-4d2a-a6c7-8eda11ee753e,d4d35d5b-4c8c-4a2e-ac28-f7eec85e7583,2024-10-07,2024-10-07 02:33:39.855
4,5,183b7feb-3cb1-4d13-b9ee-2b7730b7e5fa,d15a7fd3-0ac5-4e98-b898-b1a07e986c79,2024-10-07,2024-10-07 03:01:55.512
...,...,...,...,...,...
33510,32904,NaN,864ef502-c5c5-4cea-a819-34aa03d9cc8d,2025-02-11,2025-02-11 19:32:07.695
33511,32965,2efce7cb-0650-458a-919b-2521ebf5217d,6fb32f41-5a88-4c01-b957-7e018486bc34,2025-02-12,2025-02-12 01:03:43.426
33512,32967,30b97efd-c507-4dd0-819f-feca3f116a53,ec97eb51-b5e1-4885-a63d-2701f2685525,2025-02-12,2025-02-12 01:05:58.007
33513,33047,NaN,e91a5c48-61be-474a-8ad6-93c00402c5ed,2025-02-12,2025-02-12 05:00:43.135


In [25]:
df['access_date'] = pd.to_datetime(df['access_date'], format='mixed')
df['created_at'] = pd.to_datetime(df['created_at'], format='mixed')

In [219]:
df.drop(df[df['user_id'].isna()].index, inplace=True)

In [231]:
temp_df = pd.DataFrame(df.groupby('access_date')['user_id'].count())
temp_df['access_date'] = temp_df.index
temp_df.reset_index(drop=True, inplace=True)
temp_df['access_date'] = pd.to_datetime(temp_df['access_date'])

con1 = temp_df['access_date'] >= datetime(2024, 11, 4)
con2 = temp_df['access_date'] < datetime(2025, 1, 27)

temp_df = temp_df[con1 & con2].sort_values(by='access_date', ascending=True)
temp_df

,user_id,access_date
28,226,2024-11-04
29,215,2024-11-05
30,218,2024-11-06
31,191,2024-11-07
32,216,2024-11-08
...,...,...
107,217,2025-01-22
108,219,2025-01-23
109,398,2025-01-24
110,180,2025-01-25


In [245]:
dau_df = pd.DataFrame(temp_df.groupby(pd.Grouper(key='access_date', freq='1D'))['user_id'].unique().sum())
dau_df

,0
0,18482


In [235]:
wau_df = pd.DataFrame(temp_df.groupby(pd.Grouper(key='access_date', freq='W'))['user_id'].sum().reset_index())
wau_df

,access_date,user_id
0,2024-11-10,1407
1,2024-11-17,1705
2,2024-11-24,1436
3,2024-12-01,1357
4,2024-12-08,1382
5,2024-12-15,1595
6,2024-12-22,1461
7,2024-12-29,1409
8,2025-01-05,1533
9,2025-01-12,1680


In [237]:
mau_df = pd.DataFrame(temp_df.groupby(pd.Grouper(key='access_date', freq='1M'))['user_id'].sum().reset_index())
mau_df

/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_64214/1280596547.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  mau_df = pd.DataFrame(temp_df.groupby(pd.Grouper(key='access_date', freq='1M'))['user_id'].sum().reset_index())


,access_date,user_id
0,2024-11-30,5723
1,2024-12-31,6433
2,2025-01-31,6326


In [298]:
df['access_date'] = pd.to_datetime(df['access_date'])
con1 = df['access_date'] >= datetime(2024, 12, 1)
con2 = df['access_date'] < datetime(2025, 1, 1)
df.loc[con1 & con2, 'user_id'].unique().size

1835

In [314]:
con = df['user_id'].isna()
temp_df = df[con]
temp_df

,id,user_id,device_id,access_date,created_at
15,15,NaN,f8875b37-0b76-426e-9754-c4ea115f2f16,2024-10-07,2024-10-07 10:22:24.861
22,23,NaN,e4920ff6-b5bb-4a3c-bc0c-d6c1c8313074,2024-10-08,2024-10-08 05:47:55.023
32,33,NaN,afa5f2d3-cc67-4b5b-b82b-fb0ff6f3d0b5,2024-10-08,2024-10-08 06:15:44.391
33,34,NaN,39cb3a0b-6efd-4769-9ac8-3e669c06372c,2024-10-08,2024-10-08 06:24:49.333
34,35,NaN,ccfc1b45-dd96-4c32-ba88-d1f46276c080,2024-10-08,2024-10-08 06:27:01.411
...,...,...,...,...,...
33491,31953,NaN,f1d05369-af18-469c-895f-a8b6b20f0fbe,2025-02-09,2025-02-09 23:27:38.475
33502,32353,NaN,d75d751b-0793-49e1-83a4-9ff9f1235a7b,2025-02-11,2025-02-11 03:25:34.195
33506,32698,NaN,ffefd7b8-1945-4a58-a052-2dd69f802914,2025-02-11,2025-02-11 06:51:38.905
33510,32904,NaN,864ef502-c5c5-4cea-a819-34aa03d9cc8d,2025-02-11,2025-02-11 19:32:07.695


In [316]:
temp_df['device_id'].duplicated()

15       False
22       False
32       False
33       False
34       False
         ...  
33491    False
33502    False
33506    False
33510     True
33513    False
Name: device_id, Length: 6011, dtype: bool